<a href="https://colab.research.google.com/github/chvn00/sistemas_embebidos/blob/main/MLP_proy_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final - Parte MLP
# Profesor: Cesar Hernando Valencia Niño
# Sistemas Embebidos

## Entrenamiento del Modelo de IA para la Línea de Producción Inteligente
Generación del dataset sintético y entrenamiento de una red neuronal MLP embebible en Raspberry Pi

En este cuaderno se desarrolla el proceso completo de construcción, entrenamiento y extracción de pesos de una red neuronal del tipo MLP (Multilayer Perceptron) que luego será implementada directamente en una Raspberry Pi como parte de una mini línea de producción inteligente. Este modelo será el encargado de tomar decisiones automáticas basadas en varias lecturas de sensores.

## Arquitectura del modelo neuronal

La red que entrenaremos tiene una arquitectura 4 → 32 → 3, estructurada de la siguiente manera:

4 neuronas de entrada:
Representan las cuatro variables del proceso industrial simulado:

- Distancia (posición de la pieza).

- Luz (reflectancia/color) para estimar si la pieza está dentro de una tolerancia visual.

- Temperatura, simulando un horno o motor industrial.

- Modo de operación, codificado a partir de un DIP switch (tipo de producto o nivel de exigencia).

1 capa oculta con 8 neuronas y función de activación ReLU, que le da capacidad no lineal para separar regiones complejas del espacio de entrada.

3 neuronas de salida, correspondientes a las tres decisiones del sistema:

- Clase 0: Producto Aceptado (estado normal)

- Clase 1: Producto en Revisión (advertencia)

- Clase 2: Producto Rechazado / Condición crítica

Este modelo es suficientemente simple para correr en tiempo real dentro de la Raspberry Pi, pero lo bastante flexible para tomar decisiones útiles basadas en múltiples condiciones simultáneas.

## Dataset sintético: simulación del proceso industrial

Como no disponemos de datos reales de una fábrica, se genera un dataset sintético realista, donde cada muestra representa una "pieza" pasando por una estación de inspección. Para cada pieza se simulan:

- Distancia: si la pieza está bien posicionada (8–12 cm) o demasiado cerca/lejos (fallo).

- Luz: si la superficie refleja el nivel esperado (prueba visual).

- Temperatura: si el proceso térmico estuvo dentro del rango normal o presenta enfriamiento/sobrecalentamiento.

- Modo: define tolerancias más o menos estrictas según el tipo de producto.

Con estas variables se define una regla lógica industrial que clasifica cada pieza como normal, advertencia o crítica según sus desviaciones. Esta regla es desconocida para la red neuronal, pero se usa para generar las etiquetas del dataset. Luego el modelo aprenderá a aproximar este comportamiento.

El dataset final contiene miles de muestras distribuidas entre los tres estados posibles, lo que permite un entrenamiento estable y generalizable.

## Objetivo del entrenamiento

El objetivo es entrenar un modelo capaz de:

- Recibir valores normalizados de distancia, luz, temperatura y modo.

- Procesarlos mediante el MLP para obtener una predicción.

- Clasificar cada pieza en NORMAL, ADVERTENCIA o CRÍTICO.

- Poder exportarse a la Raspberry Pi, donde el modelo será ejecutado sin scikit-learn mediante únicamente operaciones matriciales (multiplicaciones y sumas).

## Qué se obtiene al finalizar este Colab

Al final del cuaderno tendrás:

- Un dataset sintético realista que representa condiciones industriales.

- Una red neuronal entrenada para clasificar piezas en una mini línea de producción.

  - Las matrices de pesos:

  - W1, b1 para la capa oculta.

  - W2, b2 para la capa de salida.

- Estas matrices estarán impresas en pantalla para copiarse al código embebido de la Raspberry Pi.

- Un modelo listo para usarse en inferencia en tiempo real, controlando LEDs, alertas y acciones del sistema físico.

In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

np.random.seed(42)

# ============================================
# 2. Generador de datos sintéticos
#    Simula una mini línea de producción
# ============================================

def generar_muestra():
    """
    Genera una sola muestra sintética del proceso.

    Variables "físicas":
      - distancia (cm):  5 a 20 aprox.
      - luz (0-1):       intensidad/reflectancia
      - temperatura (°C): 20 a 90 aprox.
      - modo (0,1,2,3):  tipo de producto / modo de operación

    Retorna:
      x_norm: vector de 4 features normalizadas en [0,1]
      y: clase (0 = NORMAL, 1 = ADVERTENCIA, 2 = CRÍTICO)
    """

    rng = np.random.default_rng()

    # --- 1) Elegimos modo de operación (0..3) ---
    modo = rng.integers(0, 4)  # 0,1,2,3

    # --- 2) Definimos un "estado base" casi ideal ---
    # Ideal: pieza bien posicionada, iluminación correcta, temperatura razonable
    dist = rng.normal(10, 1.0)     # ideal ~10 cm
    luz  = rng.normal(0.6, 0.1)    # ideal ~0.6 (0-1)
    temp = rng.normal(50, 5.0)     # ideal ~50 °C

    # --- 3) A veces introducimos defectos / desviaciones ---
    # Probabilidad de que haya "problemas" en alguna variable
    if rng.random() < 0.5:
        # Escogemos una o dos variables para deformar
        n_vars_defect = rng.integers(1, 3)
        vars_defect = rng.choice(['dist', 'luz', 'temp'], size=n_vars_defect, replace=False)

        for v in vars_defect:
            if v == 'dist':
                # pieza demasiado cerca o demasiado lejos
                if rng.random() < 0.5:
                    dist = rng.uniform(4, 6)   # demasiado cerca
                else:
                    dist = rng.uniform(16, 20) # demasiado lejos
            elif v == 'luz':
                # pieza muy oscura o muy clara
                if rng.random() < 0.5:
                    luz = rng.uniform(0.0, 0.25)
                else:
                    luz = rng.uniform(0.8, 1.0)
            elif v == 'temp':
                # horno / motor muy frío o sobrecalentado
                if rng.random() < 0.5:
                    temp = rng.uniform(20, 35)
                else:
                    temp = rng.uniform(70, 90)

    # --- 4) Recortamos a rangos razonables ---
    dist = float(np.clip(dist, 4, 20))
    luz  = float(np.clip(luz, 0.0, 1.0))
    temp = float(np.clip(temp, 20, 90))

    # --- 5) Definimos una regla "industrial" para la clase destino (etiqueta) ---
    # Calculamos penalizaciones por desviaciones
    penalty = 0

    # Distancia: ideal 8-12, aceptable 7-14
    if dist < 7 or dist > 14:
        penalty += 1
    if dist < 6 or dist > 16:
        penalty += 1  # penalización adicional si está muy mal

    # Luz: ideal 0.4-0.7, aceptable 0.3-0.8
    if luz < 0.3 or luz > 0.8:
        penalty += 1
    if luz < 0.2 or luz > 0.9:
        penalty += 1

    # Temperatura: ideal 45-55, aceptable 40-65
    if temp < 40 or temp > 65:
        penalty += 1
    if temp < 35 or temp > 75:
        penalty += 1

    # El modo puede hacer más estricta la inspección para algunos productos
    # Por ejemplo, modo 3 es un producto "premium" con control más estricto
    if modo == 3 and (temp < 45 or temp > 60):
        penalty += 1

    # Asignamos clase según penalización total
    # 0 = NORMAL, 1 = ADVERTENCIA, 2 = CRÍTICO
    if penalty <= 1:
        y = 0
    elif penalty == 2:
        y = 1
    else:
        y = 2

    # --- 6) Normalización de las entradas para el MLP (muy importante para replicar en la Pi) ---
    # Definimos normalizaciones simples, entre 0 y 1:
    # Nota: estas fórmulas se deben usar también en la Raspberry Pi.
    dist_norm = (dist - 4.0) / (20.0 - 4.0)       # 4..20 -> 0..1
    temp_norm = (temp - 20.0) / (90.0 - 20.0)     # 20..90 -> 0..1
    luz_norm  = luz                                # ya está en 0..1
    modo_norm = modo / 3.0                        # 0..3 -> 0..1

    x_norm = np.array([dist_norm, luz_norm, temp_norm, modo_norm], dtype=np.float32)

    return x_norm, y


def generar_dataset(samples_per_class=1000):
    """
    Genera un dataset balanceado, con `samples_per_class` muestras por cada clase.
    X: matriz (N, 4)
    y: vector (N,)
    """
    X_list, y_list = [], []
    class_counts = [0, 0, 0] # Initialize counts for classes 0, 1, 2

    while any(c < samples_per_class for c in class_counts):
        x, label = generar_muestra()
        if class_counts[label] < samples_per_class:
            X_list.append(x)
            y_list.append(label)
            class_counts[label] += 1

    X = np.vstack(X_list)
    y = np.array(y_list, dtype=int)

    return X, y


# ============================================
# 3. Crear dataset y explorar distribución
# ============================================

X, y = generar_dataset(samples_per_class=1000) # Modified call to generate a balanced dataset
print("Dimensiones de X:", X.shape)   # (N, 4)
print("Dimensiones de y:", y.shape)   # (N,)

print("\nPrimeras 5 filas de X:\n", X[:5])
print("\nPrimeras 5 etiquetas de y:\n", y[:5])

unique, counts = np.unique(y, return_counts=True)
print("Distribución de clases (0=Normal, 1=Advertencia, 2=Crítico):")
for cls, cnt in zip(unique, counts):
    print(f"  Clase {cls}: {cnt} muestras ({cnt/len(y)*100:.1f}%)")

# ============================================
# 4. Separar en entrenamiento y prueba
# ============================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=0, stratify=y
)

print("Train:", X_train.shape, " Test:", X_test.shape)

# ============================================
# 5. Definir y entrenar la MLP
#    Arquitectura ejemplo: 4 -> 16 -> 3 (Updated)
# ============================================

mlp = MLPClassifier(
    hidden_layer_sizes=(32,),    # una capa oculta de 16 neuronas (Updated)
    activation='relu',
    solver='adam',
    max_iter=5000,               # Incrementado el número de iteraciones (Updated)
    random_state=0
)

mlp.fit(X_train, y_train)

# ============================================
# 6. Evaluación
# ============================================
y_pred = mlp.predict(X_test)

print("\nAccuracy en prueba:", accuracy_score(y_test, y_pred))
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred, digits=3))

print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

# ============================================
# 7. Extraer pesos del MLP para la Raspberry Pi
#    coefs_[0]: (n_features, n_hidden) -> (4, 16)
#    coefs_[1]: (n_hidden, n_outputs) -> (16, 3)
# ============================================

W1 = mlp.coefs_[0]       # (4, 16) - Updated shape
b1 = mlp.intercepts_[0]  # (16,) - Updated shape
W2 = mlp.coefs_[1]       # (16, 3) - Updated shape
b2 = mlp.intercepts_[1]  # (3,)

# Custom formatter for floats to ensure 5 decimal places and no suppression
float_formatter = lambda x: "%.5f" % x

print("\n=== Pesos para implementar en la Raspberry Pi ===")
print("W1.shape:", W1.shape)
print("W1 = np.array(")
print(np.array2string(W1, separator=', ', formatter={'float_kind':float_formatter}))
print(")")
print("\nb1.shape:", b1.shape)
print("b1 = np.array(")
print(np.array2string(b1, separator=', ', formatter={'float_kind':float_formatter}))
print(")")

print("\nW2.shape:", W2.shape)
print("W2 = np.array(")
print(np.array2string(W2, separator=', ', formatter={'float_kind':float_formatter}))
print(")")
print("\nb2.shape:", b2.shape)
print("b2 = np.array(")
print(np.array2string(b2, separator=', ', formatter={'float_kind':float_formatter}))
print(")")


Dimensiones de X: (3000, 4)
Dimensiones de y: (3000,)

Primeras 5 filas de X:
 [[0.4411  0.5004  0.13894 0.33333]
 [0.34287 0.50525 0.06516 0.66667]
 [0.36516 0.70789 0.39292 0.     ]
 [0.2725  0.53403 0.3314  0.33333]
 [0.37947 0.55869 0.43881 0.66667]]

Primeras 5 etiquetas de y:
 [1 1 0 0 0]
Distribución de clases (0=Normal, 1=Advertencia, 2=Crítico):
  Clase 0: 1000 muestras (33.3%)
  Clase 1: 1000 muestras (33.3%)
  Clase 2: 1000 muestras (33.3%)
Train: (2250, 4)  Test: (750, 4)

Accuracy en prueba: 0.9453333333333334

Reporte de clasificación:
              precision    recall  f1-score   support

           0      0.992     0.964     0.978       250
           1      0.916     0.920     0.918       250
           2      0.930     0.952     0.941       250

    accuracy                          0.945       750
   macro avg      0.946     0.945     0.946       750
weighted avg      0.946     0.945     0.946       750

Matriz de confusión:
[[241   9   0]
 [  2 230  18]
 [  0  12 23